In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## KEGGapFiller Example

In [1]:
from refinegems.classes.gapfill import KEGGapFiller
from refinegems.utility.io import load_model

modelpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435.xml'
model = load_model(modelpath,'libsbml')
cmodel = load_model(modelpath,'cobra')

print(len(model.getPlugin(0).getListOfGeneProducts()))

gfk = KEGGapFiller('sha')

gfk_missing_genes = gfk.find_missing_genes(model)

gfk_missing_reac = gfk.find_missing_reacs(cmodel,gfk_missing_genes)

model = gfk.fill_model(model,gfk_missing_genes,gfk_missing_reac)

print(len(model.getPlugin(0).getListOfGeneProducts()))

/Users/brune/miniconda3/envs/sprg/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


787
Running in debugging mode.


100%|██████████| 20/20 [00:24<00:00,  1.23s/it]


fill_model: Running in debugging mode


Adding genes to model: 100%|██████████| 2/2 [00:00<00:00, 3636.15it/s]

789


In [2]:
gfk._statistics 

{'genes': {'missing (before)': 20, 'added': 2, 'missing (after)': 18},
 'reactions': {'added (total)': 7, 'failed to build': 3},
 'metabolites': {}}

## GeneGapFiller Example

In [1]:
from refinegems.classes.gapfill import GeneGapFiller
from refinegems.utility.io import load_model

modelpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435.xml'
model = load_model(modelpath,'libsbml')
cmodel = load_model(modelpath,'cobra')

gffpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435_RefSeq.gff'

tfasta = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435_proteins_genome.fasta'
spdb = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/swissprot.dmnd'
spmap = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/uniprot_table.tsv'
kwargs = {'outdir':'/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435',
          'sens':'more-sensitive',
          'cov':90.0,
          't':4,
          'pid':90.0}

gf2 = GeneGapFiller()
gf2_missing_genes = gf2.find_missing_genes(gffpath,model)
# ncbiprotein | locus_tag | ec-code
mapped_res = gf2.find_missing_reacs(model=cmodel,
                                   missing_genes=gf2_missing_genes,
                                   fasta=tfasta, 
                                   dmnd_db=spdb,
                                   swissprot_map=spmap,
                                   **kwargs)

model = gf2.fill_model(model,mapped_res[0],mapped_res[1],formula_check='existence')
# formula_check,exclude_dna,exclude_rna,idprefix,namespace

/Users/brune/miniconda3/envs/sprg/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


Running in debugging mode.
fill_model: Running in debugging mode


Adding genes to model: 100%|██████████| 3/3 [00:00<00:00, 5345.33it/s]


In [2]:
gf2._statistics
# possible problem with the current stats: some stuff might be counted twice or more, since the
# current implementation depends on the tables (esp. reactions, since ID depends on NCBI 
# protein accession number and not the uniqueness of the ID)


{'genes': {'missing (before)': 1800,
  'added': 3,
  'missing (after)': 4,
  'no locus tag': 108},
 'reactions': {'added (total)': 7,
  'failed to build': 3,
  'no NCBI, no EC': 0,
  'NCBI, no EC': 43},
 'metabolites': {}}

## BioCycGapFiller example

In [1]:
from refinegems.classes.gapfill import BioCycGapFiller
from refinegems.utility.io import load_model

modelpath = './test_files/ATCC29970.xml'
model = load_model(modelpath,'libsbml')
cmodel = load_model(modelpath,'cobra')

gffpath = '/Users/doebel/Desktop/Master_thesis/Repository/Shaemolyticus/data/ATCC29970/ATCC29970_RefSeq.gff'
biocyc_gene_tbl_path = '/Users/doebel/Desktop/Master_thesis/Repository/Shaemolyticus/archive/master_thesis_ishgog23/data/ATCC29970/ATCC29970_Accession-22Reactions.txt'
biocyc_reacs_tbl_path = './test_files/ATCC29970_biocyc_rxns.txt'
fasta = '/Users/doebel/Desktop/Master_thesis/Repository/Shaemolyticus/data/ATCC29970/ATCC29970_proteins_genome.fasta'

gfbc = BioCycGapFiller(biocyc_gene_tbl_path, biocyc_reacs_tbl_path, gffpath)
gfbc_missing_genes = gfbc.find_missing_genes(model)
# ncbiprotein | locus_tag | ec-code
mapped_res = gfbc.find_missing_reacs(cmodel)

#model = gfbc.fill_model(model,mapped_res[0],mapped_res[1],formula_check='existence')
# formula_check,exclude_dna,exclude_rna,idprefix,namespace

/Users/doebel/miniconda3/envs/refineGEMs/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


In [2]:
gfbc.missing_reacs

,add_to_GPR,ec-code,equation,id,ncbiprotein,reference,via
0,[3HAD80],[4.2.1.59],(3R)-3-hydroxyoctanoyl-[acp] -> (2E)-oct-2-e...,4.2.1.59-RXN,[WP_011275247.1],None,BioCyc
1,[AMMQLT8],[2.1.1.163],S-adenosyl-L-methionine + demethylmenaquinol-8...,ADOMET-DMK-METHYLTRANSFER-RXN,[WP_011275738.1],None,BioCyc
2,"[NTP1, ATPM]","[3.6.1.3, 3.6.1.15]",ATP + H2O -> ADP + phosphate + H+,ATPASE-RXN,[WP_016931380.1],None,BioCyc
3,[LDH_D],[1.1.1.28],(R)-lactate + NAD+ <- pyruvate + NADH + H+,DLACTDEHYDROGNAD-RXN,[WP_011274818.1],None,BioCyc
4,[DHNAOT4],[2.5.1.74],"all-trans-octaprenyl diphosphate + 1,4-dihydro...",DMK-RXN,[WP_016931372.1],None,BioCyc
...,...,...,...,...,...,...,...
233,None,NaN,1 MNXM11@MNXD1 + 1 MNXM167418@MNXD1 + 1 MNXM72...,{MNXR124075},[WP_011276231.1],{'metacyc.reaction': 'TRYPTOPHAN--TRNA-LIGASE-...,MetaNetX
234,None,NaN,1 MNXM11@MNXD1 + 1 MNXM16655@MNXD1 + 1 MNXM728...,{MNXR124080},[WP_011275493.1],{'metacyc.reaction': 'TYROSINE--TRNA-LIGASE-RX...,MetaNetX
235,None,NaN,1 MNXM1104679@MNXD1 + 1 MNXM1446@MNXD1 + 1 MNX...,{MNXR146695},[WP_011275264.1],{'metacyc.reaction': 'UDP-NACMURALGLDAPAALIG-R...,MetaNetX
236,None,NaN,1 MNXM1101258@MNXD1 = 1 MNXM1104529@MNXD1,{MNXR151539},[WP_033079611.1],"{'metacyc.reaction': 'UDPGLCNACEPIM-RXN', 'ori...",MetaNetX


In [3]:
gfbc.manual_curation['BioCyc reactions unmappable']

,add_to_GPR,ec-code,equation,id,ncbiprotein,reference,via
238,None,NaN,"(1,4-alpha-D-galacturonosyl)(n+m) -> (1,4-al...",4.2.2.2-RXN,[WP_011275342.1],{'ec-code': ['4.2.2.2']},BioCyc
239,None,NaN,a [protein]-L-proline (omega = 180) <--> a [...,PEPTIDYLPROLYL-ISOMERASE-RXN,"[WP_016931161.1, WP_011275422.1, WP_016930839.1]",{'ec-code': ['5.2.1.8']},BioCyc


In [4]:
gfbc._statistics

{'genes': {'missing (before)': 235,
  'duplicates': 0,
  'added': 0,
  'missing (after)': 0,
  'missing (unmappable)': 0},
 'reactions': {'added (total)': 0,
  'failed to build': 0,
  'missing (before)': 240,
  'add to GPR (BioCyc)': 11,
  'mapped2MNX': 227,
  'mapped2BiGG': 0,
  'remaining_unmapped': 2},
 'metabolites': {}}

In [5]:
gfbc.manual_curation['BioCyc genes unmappable']

,locus_tag,id


#### Get new BioCyc reactions table without BioCyc access

In [16]:
import pandas as pd

df_with_new_info = pd.read_csv('./test_files/All-reactions-of-S.-haemolyticus-ATCC-29970.txt', sep='\t')
df_with_new_info.drop(['Object ID.1', 'EC-Number', 'Products of reaction', 'Pathways of a reaction'], axis=1, inplace=True)
df_old = pd.read_csv('/Users/doebel/Desktop/Master_thesis/Repository/Shaemolyticus/archive/master_thesis_ishgog23/data/ATCC29970/ATCC29970_reactions.txt', sep='\t')
df_old.rename({'Reaction': 'Object ID'}, axis=1, inplace=True)
df_old.drop('Accession-2', axis=1, inplace=True)

df_new = df_with_new_info.merge(df_old, how='left', on='Object ID')
df_new.to_csv('./test_files/ATCC29970_biocyc_rxns.txt', index=False, sep='\t')
df_new

,Reaction,Object ID,Substrates (reactants and products) of a reaction,Reactants of reaction,Products of reaction,EC-Number,Reaction-Direction,Spontaneous?
0,S-adenosyl-L-methionine + demethylmenaquinol-7...,RXN-9191,demethylmenaquinol-7 // <i>S</i>-adenosyl-L-me...,S-ADENOSYLMETHIONINE // CPD-12117,CPD-12125 // ADENOSYL-HOMO-CYS // PROTON,EC-2.1.1.163,PHYSIOL-LEFT-TO-RIGHT,NaN
1,"a 2,3,4-saturated (3R)-3-hydroxyacyl-CoA <-->...",OHBUTYRYL-COA-EPIM-RXN,"a 2,3,4-saturated (3<i>R</i>)-3-hydroxyacyl-Co...",D-3-HYDROXYACYL-COA,L-3-HYDROXYACYL-COA,EC-5.1.2.3,REVERSIBLE,NaN
2,"glycine + a tetrahydrofolate + NAD+ -> a 5,1...",GCVMULTI-RXN,NAD<sup>+</sup> // a tetrahydrofolate // glyci...,GLY // THF-GLU-N // NAD,METHYLENE-THF-GLU-N // AMMONIUM // CARBON-DIOX...,EC-1.4.1,LEFT-TO-RIGHT,NaN
3,N-acetyl-beta-neuraminate -> N-acetyl-beta-D...,RXN-15313,<i>N</i>-acetyl-&beta;-neuraminate // <i>N</i>...,CPD0-1123,N-ACETYL-D-MANNOSAMINE // PYRUVATE,EC-4.1.3.3,PHYSIOL-LEFT-TO-RIGHT,NaN
4,D-xylulose + ATP -> D-xylulose 5-phosphate +...,XYLULOKIN-RXN,ATP // D-xylulose // H<SUP>+</SUP> // D-xylulo...,D-XYLULOSE // ATP,PROTON // XYLULOSE-5-PHOSPHATE // ADP,EC-2.7.1.17,LEFT-TO-RIGHT,NaN
...,...,...,...,...,...,...,...,...
1064,(3R)-3-hydroxydecanoyl-[acp] -> (2E)-dec-2-e...,RXN-9655,a (3<i>R</i>)-3-hydroxydecanoyl-[acp] // a (2<...,Beta-hydroxydecanoyl-ACPs,Trans-D2-decenoyl-ACPs // WATER,EC-4.2.1.59,LEFT-TO-RIGHT,NaN
1065,2 L-arginine + 3 NADPH + H+ + 4 oxygen -> 2 ...,NITRIC-OXIDE-SYNTHASE-RXN,oxygen // H<SUP>+</SUP> // NADPH // L-arginine...,ARG // NADPH // PROTON // OXYGEN-MOLECULE,L-CITRULLINE // NITRIC-OXIDE // NADP // WATER,EC-1.14.13.39,PHYSIOL-LEFT-TO-RIGHT,NaN
1066,a [protein]-Npi-phospho-L-histidine + 2-phosph...,RXN-15510,2-phospho-D-glycerate // a [protein]-<i>N<sup>...,Protein-pi-phospho-L-histidines // 2-PG,Protein-Histidines // 23-DIPHOSPHOGLYCERATE,NaN,REVERSIBLE,NaN
1067,(5Z)-tetradecenoyl-CoA + acetyl-CoA <- 3-oxo...,RXN-17782,acetyl-CoA // (5Z)-tetradecenoyl-CoA // 3-oxo-...,CPD-19167 // CO-A,CPD-15436 // ACETYL-COA,EC-2.3.1.16,PHYSIOL-RIGHT-TO-LEFT,NaN


## the "Filling" part of Gapfilling

**input**

- the model
- missing genes table
- missing reacs table

**out**

- the extended model

**else**

- logging
- save stats information

### Further ideas and Code snippets for the filling part

missing stuff:
- parse BioCyc reaction string
- build biocyc_metabolite
- build biocyc_reaction

### Ideas for collect the stats information

In [7]:
len(cmodel.reactions)

1538